In [ ]:
### alignment comparison dependencies 

import pandas as pd
import ast
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.tag.stanford import StanfordPOSTagger
from nltk.util import ngrams
from collections import Counter
import spacy, benepar
import math
import random
import numpy as np 
import re
from apted import APTED, apted, single_path_functions, node_indexer, resources,helpers
from scipy import spatial
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))

In [ ]:
def cont_parse(utt):
    try:
        return list(nlp(utt).sents)[0]._.parse_string
    except:
        return ''
    
    
def clean_str(string):
    return ''.join(e for e in string if e.isalnum())



def cont_tree_nodes_only(tree_parse_string):
    
    if tree_parse_string == '()':
        return '()'
    
    else:
    
        start = []
        end = []
        ch = ''
        ts = ''.join(list(reversed(tree_parse_string)))
        for i, x in enumerate(ts):
            ch += x
            if i != 0:

                if (x != ')') and (ts[i-1] == ')'):
                    start.append(i)
                    count = i
                    for y in ts[i:]:
                        count +=1
                        if y == ' ':
                            end.append(count)
                            break

        cords = list(zip(start, end))


        s = ''

        count = 0

        for cord in cords:

            count += 1

            if count > 1:
                seq_len = len(ts[cord[0]: cord[1]])

                seq_ = ''.join(['_' for x in range(seq_len)])

                s = s[:cord[0]] + seq_ + s[cord[1]:]

            else:


                seq_len = len(ts[cord[0]: cord[1]])

                seq_ = ''.join(['_' for x in range(seq_len)])



                s = ts[:cord[0]] + seq_ + ts[cord[1]:]

        return ''.join(list(reversed(s.replace('_', ''))))
    
def remove_punct_nodes(s):

    punct_li = ['(``)', '('')',  '(,)',  '(.)', '(!)', '(?)', '(:)', '(;)', '(-)',
               '(''))', "(''))", '(.))', '(!))',  '(?))']
    
    end_punct = ['(''))', "(''))", '(.))', '(!))',  '(?))']
    
    s2= []
    
    count = 0

    for x in s.split():
        if x in end_punct:
            count+= 1
        if x not in punct_li:
            s2.append(x)
            
    for i in range(count):
        s2.append(')')
    
    return ''.join(s2)

def js_syntax_convert(s):
    return s.replace(')', ']').replace('(', '[')
    

def buildVector(iterable1, iterable2):
    
    counter1 = Counter(iterable1)
    counter2= Counter(iterable2)
    all_items = set(counter1.keys()).union( set(counter2.keys()) )
    vector1 = [counter1[k] for k in all_items]
    vector2 = [counter2[k] for k in all_items]
    return vector1, vector2

def pre_traversal_str(s):
    return [x.split()[0] for x in [x.strip() for x in re.split('[()]', s) if x != ''] if x != ''][:]
                
                
def tree_depth(s):
    li = re.split('[)]', s)
    li = [x.strip() for x in li]
    for i, x in enumerate(li):
        if x == '':                       
            if set(['']) == set(li[i:]):

                if len(li[i:]) == 1:
                    return 0
                else: 
                    if set(['']) == set(li[i:]):
                    
                
                        return len(li[i:])
            
            else:
                continue
                
        else:
            continue



def apt_edit_tree_dist_format(tree_string):
    return tree_string.replace('(', '{').replace(')', '}').replace(' ', '')


def apt_edit_tree_dist_format_bt(tree_string, bracket_type1, bracket_type2):
    return tree_string.replace(bracket_type1, '{').replace(bracket_type2, '}')


def count_sub_trees_root(string_tree):
    
    string_tree = apt_edit_tree_dist_format(string_tree)
    
    num_sub_trees = 0
    
    try:
    
        for i, x in enumerate(string_tree):

            if (x == '}') and  (string_tree[i+1] == '}') and (string_tree[i+2] == '}'):
                num_sub_trees += 1
                break

            else:
                if (x == '}') and  (string_tree[i+1] == '}'):
                    num_sub_trees += 1

        return num_sub_trees
    except:
        return get_all_sub_trees_count(string_tree)[1]


def get_sub_trees_from_root(str_tree):
    
    str_tree = apt_edit_tree_dist_format(str_tree)
    
    
    sub_tree_preorder_strings = []
    start_sub_tree_indices = []
    end_sub_tree_indices = []
    start_sub_tree_indices.append(0)
    
    try:
        for i, x in enumerate(str_tree):


            if (x == '}') and  (str_tree[i+1] == '}') and (str_tree[i+2] == '}'):
                end_sub_tree_indices.append(i+1)
                break

            else:
                if (x == '}') and  (str_tree[i+1] == '}'):
                    end_sub_tree_indices.append(i+1)
                    start_sub_tree_indices.append(i+1)

        for cord in list(zip(start_sub_tree_indices, end_sub_tree_indices)):

            sub_tree_preorder_strings.append(str_tree[cord[0]:cord[1]])

        sub_tree_preorder_strings = [x.replace('}', ' ').replace('{', ' ').replace('  ', ' ').strip() for x in sub_tree_preorder_strings]

        return sub_tree_preorder_strings
    
    except:
        return get_all_sub_trees_count(str_tree)[0]
                



def get_all_sub_trees_count(str_tree):
    sub_trees = [x.replace('}', ' ').replace('{', ' ').strip()  for x in apt_edit_tree_dist_format(str_tree).split('}{')]
    num_sub_trees = len(sub_trees)
    
    return [sub_trees, num_sub_trees]


def compare_all_sub_trees(tree1, tree2):
    
    num_shared_sub_trees = 0
    
    prop_sub_tree_shared = 0
    
    prop_sub_tree = 0
    
    prop_sub_trees_li = []
    
    tree1_preorder_subtrees = get_all_sub_trees_count(tree1)[0]
    tree2_preorder_subtrees = get_all_sub_trees_count(tree2)[0]
    
    tree1_preorder_subtrees_num = get_all_sub_trees_count(tree1)[1]
    tree2_preorder_subtrees_num = get_all_sub_trees_count(tree2)[1]
    
    tree1_indices = [i for i in range(len(tree1_preorder_subtrees))]
    
    for ix, x in enumerate(tree2_preorder_subtrees):
        if x in tree1_preorder_subtrees:

            num_shared_sub_trees += 1
            
        else:

            if ix in tree1_indices:

                if x != tree1_preorder_subtrees[ix]:
                    for e in x.split():
                        if e in tree1_preorder_subtrees[ix].split():

                            prop_sub_tree += 1 

    
    try:                
        prop_sub_trees_shared = prop_sub_tree/sum([len(x.split()) for x in tree1_preorder_subtrees])
        
    except:
        prop_sub_trees_shared = 0
                     
    
    return [num_shared_sub_trees, prop_sub_trees_shared]



def compare_root_sub_trees(tree1, tree2):
    
    num_shared_sub_trees = 0
    
    prop_sub_tree_shared = 0
    
    prop_sub_tree = 0
    
    prop_sub_trees_li = []
    
    tree1_preorder_subtrees = get_sub_trees_from_root(tree1)
    tree2_preorder_subtrees = get_sub_trees_from_root(tree2)
    
    tree1_indices = [i for i in range(len(tree1_preorder_subtrees))]
    
    for ix, x in enumerate(tree2_preorder_subtrees):
        if x in tree1_preorder_subtrees:

            num_shared_sub_trees += 1
            
        else:

            if ix in tree1_indices:
                if x != tree1_preorder_subtrees[ix]:
                    for e in x.split():
                        if e in tree1_preorder_subtrees[ix].split():

                            prop_sub_tree += 1 
                            
                            
    try:                
        prop_sub_trees_shared = prop_sub_tree/sum([len(x.split()) for x in tree1_preorder_subtrees])
        
    except:
        prop_sub_trees_shared = 0
    
    return [num_shared_sub_trees, prop_sub_trees_shared]
    
    
def compare_preorder_traversal_differences(preorder_traversed_tree1, preordered_traversed_tree2):
    
    deviation_diff = 0
    num_remaining_differences = 0
    
    for i, x in enumerate(preorder_traversed_tree1):
        if i >= len(preordered_traversed_tree2):

    
            num_remaining_differences = len(preorder_traversed_tree1[i:])
            
            break
                
        
        elif x == preordered_traversed_tree2[i]:
            
                deviation_diff -= 1
        else:
            deviation_diff += 1
            
    
    return deviation_diff + num_remaining_differences


def get_bases_for_entropy(tree_preoder_traversals):
    bases = collections.Counter([tmp_base for tmp_base in tree_preoder_traversals])
    
    return bases

 
def estimate_shannon_entropy(tree_preoder_traversal, bases):
    
    m = len(tree_preoder_traversal)
    bases = bases

    shannon_entropy_value = 0
    for base in bases:
        # number of residues
        n_i = bases[base]
        # n_i (# residues type i) / M (# residues in column)
        p_i = n_i / float(m)
        entropy_i = p_i * (math.log(p_i, 2))
        shannon_entropy_value += entropy_i

    return shannon_entropy_value * -1



def compare_traversal_preorder_tree_deviation(preorder_traversed_tree1, preorder_traversed_tree2, tree1, tree2, tree2_edit_distance):
    
    ### num root sub trees
    ### num all sub trees
    ### compare all sub trees
    ### compare root sub trees
    ### tree order traversal
    
    
    v1 = preorder_traversed_tree1
    v2 = preorder_traversed_tree2
    
    nv = []
    
    deviation_diff = 0
    
    tree1_edit_distance = 0 
    
    tree2_edit_distance = tree2_edit_distance
    
    source_tree = 0
    
    source_tree_num_nodes = len(v1)
    
    target_tree_num_nodes = len(v2)
    
    set_diff_source_target  = len(set(v1) - (set(v2)))
    
    set_diff_target_source  = len(set(v2) - (set(v1)))
    
    tree1_num_root_sub_trees = count_sub_trees_root(tree1)
    
    tree1_num_all_sub_trees = get_all_sub_trees_count(tree1)[1]
    
    tree2_num_root_sub_trees = count_sub_trees_root(tree2)
    
    tree2_num_all_sub_trees = get_all_sub_trees_count(tree2)[1]
    
    prop_shared_all_sub_trees = compare_all_sub_trees(tree1, tree2)[1]
    
    prop_shared_root_sub_trees = compare_root_sub_trees(tree1, tree2)[1]
    
    tree1_tree2_num_shared_all_trees = compare_all_sub_trees(tree1, tree2)[0]
    
    tree1_tree2_num_shared_root_trees = compare_root_sub_trees(tree1, tree2)[0]
    
    tree2_tree1_num_shared_all_trees = compare_all_sub_trees(tree1, tree2)[0]
    
    tree2_tree1_num_shared_root_trees = compare_root_sub_trees(tree1, tree2)[0]
    
    
    

    symmetric_diff = len(set(v2).symmetric_difference(set(v1)))
    
    
    if v1 == v2:
        return {'deviation_diff': deviation_diff, 
                'source_tree':  source_tree,
                'set_diff_source_target': set_diff_source_target, 
                'set_diff_target_source': set_diff_target_source,
                'source_tree_num_nodes': source_tree_num_nodes, 
                'target_tree_num_nodes':target_tree_num_nodes,
                'tree1_edit_distance': tree1_edit_distance,
                'tree2_edit_distance': tree2_edit_distance,
                'tree1_num_root_sub_trees': tree1_num_root_sub_trees,
                'tree1_num_all_sub_trees': tree1_num_all_sub_trees,
                'tree2_num_root_sub_trees': tree2_num_root_sub_trees,
                'tree2_num_all_sub_trees': tree2_num_all_sub_trees,
                'prop_shared_all_sub_trees': prop_shared_all_sub_trees,
                'prop_shared_root_sub_trees': prop_shared_root_sub_trees,
                'tree1_tree2_num_shared_all_trees': tree1_tree2_num_shared_all_trees,
                'tree1_tree2_num_shared_root_trees': tree1_tree2_num_shared_root_trees,
                'tree2_tree1_num_shared_all_trees': tree2_tree1_num_shared_all_trees,
                'tree2_tree1_num_shared_root_trees': tree2_tree1_num_shared_root_trees
                }
    
    else:
        
        deviation_diff = compare_preorder_traversal_differences(v1, v2)
     

                    
        return {'deviation_diff': deviation_diff, 
                'source_tree':  source_tree,
                'set_diff_source_target': set_diff_source_target, 
                'set_diff_target_source': set_diff_target_source,
                'source_tree_num_nodes': source_tree_num_nodes, 
                'target_tree_num_nodes':target_tree_num_nodes,
                'tree1_edit_distance': tree1_edit_distance,
                'tree2_edit_distance': tree2_edit_distance,
                'tree1_num_root_sub_trees': tree1_num_root_sub_trees,
                'tree1_num_all_sub_trees': tree1_num_all_sub_trees,
                'tree2_num_root_sub_trees': tree2_num_root_sub_trees,
                'tree2_num_all_sub_trees': tree2_num_all_sub_trees,
                'prop_shared_all_sub_trees': prop_shared_all_sub_trees,
                'prop_shared_root_sub_trees': prop_shared_root_sub_trees,
                'tree1_tree2_num_shared_all_trees': tree1_tree2_num_shared_all_trees,
                'tree1_tree2_num_shared_root_trees': tree1_tree2_num_shared_root_trees,
                'tree2_tree1_num_shared_all_trees': tree2_tree1_num_shared_all_trees,
                'tree2_tree1_num_shared_root_trees': tree2_tree1_num_shared_root_trees}
    
    
    
def tree_sim_dev(v1, v2, tree1, tree2, tree1_depth, tree2_depth, tree2_edit_distance):
    
    tree_comp = compare_traversal_preorder_tree_deviation(v1, v2, tree1, tree2, tree2_edit_distance)
    
    tree1_depth = [tree1_depth]
    tree2_depth = [tree2_depth]
    

    source_tree_keys = ['source_tree', 'set_diff_source_target', 'source_tree_num_nodes', 
                        'tree2_tree1_num_shared_all_trees', 'tree2_tree1_num_shared_root_trees',
                       'tree1_tree2_num_shared_all_trees', 'tree1_tree2_num_shared_root_trees',
                       'tree1_num_root_sub_trees', 'tree1_num_all_sub_trees', 'tree1_edit_distance']
    
    target_tree_keys = ['deviation_diff', 'set_diff_target_source', 'target_tree_num_nodes',
                       'tree1_tree2_num_shared_root_trees', 'tree1_tree2_num_shared_all_trees',
                       'tree2_tree1_num_shared_all_trees', 'tree2_tree1_num_shared_root_trees',
                       'tree2_num_root_sub_trees', 'tree2_num_all_sub_trees', 'tree2_edit_distance']
    
    source_tree_vals = [tree_comp[k] for k in source_tree_keys]
    target_tree_vals = [tree_comp[k] for k in target_tree_keys]
    
    source_tree_node_type_counts = buildVector(v1, v2)[0]
    target_tree_node_type_counts = buildVector(v1, v2)[1]
    
    vs = buildVector(v1, v2)[0] + tree1_depth + source_tree_vals
    vt =  buildVector(v1, v2)[1] + tree2_depth + target_tree_vals

    
    r = 1 - spatial.distance.cosine(vs, vt)
    
    return {'cosine_sim': r, 'tree1_depth': tree1_depth[0], 
           'tree2_depth': tree2_depth[0],
           'source_tree_vals': source_tree_vals, 
           'target_tree_vals': target_tree_vals,
           'source_node_counts': source_tree_node_type_counts,
           'target_node_counts': target_tree_node_type_counts}


def calculate_tree_edit_distance(tree1_text, tree2_text):

    t1 = helpers.Tree.from_text(tree1_text)
    t2 = helpers.Tree.from_text(tree2_text)

    ted = APTED(t1, t2)

    return APTED.compute_edit_distance(ted)


def tree_edit_sim(tree_edit_distance):
    return 1/(1+tree_edit_distance)
    

def tree_edit_sim_sqrt(tree_edit_distance):
    
    return math.sqrt(1/(1+tree_edit_distance))


def fusarolli_align_prep(token1, token2):

    lemma1 = [x.lemma_ for x in nlp(token1)]

    lemma2 = [x.lemma_ for x in nlp(token2)]

    pos_token1 = [x.pos_ for x in nlp(token1)]

    pos_token2 = [x.pos_ for x in nlp(token2)]

    pos_lemma1 = [x.pos_ for x in nlp(' '.join(lemma1)) if str(x) != '-PRON-']

    pos_lemma2 = [x.pos_ for x in nlp(' '.join(lemma2)) if str(x) != '-PRON-']
    
    return [token1, lemma1, pos_token1, pos_lemma1, token2, lemma2, pos_token2, pos_lemma2]

def syntactic_lexical_sim(statement1, statement2):
    statement1 =[x.lower().strip() for x in statement1.split()]
    statement2 =[x.lower().strip() for x in statement2.split()]
    
    if (len(statement1) or len(statement2)) == 0:

        return [0,0,0,0]
    
    else:
    
        shared_lexical_items = 0
        num_lexical_items = len(statement1) + len(statement2)

        for word in statement1:
            if word in statement2:
                shared_lexical_items += 1

        lexical_sim_ratio = (shared_lexical_items/num_lexical_items) * 2

        nlp_statement1 = nlp(' '.join(statement1))

        nlp_statement2 = nlp(' '.join(statement2))

        statement1_pos = [x.pos_ for x in nlp_statement1]

        statement2_pos = [x.pos_ for x in nlp_statement2]

        shared_pos = 0

        num_pos =  len(statement1_pos) + len(statement2_pos)
        
        try:

            for pos in statement1_pos:
                if pos in statement2_pos:
                    shared_pos += 1

            syntactic_sim_ratio = (shared_pos/num_pos) * 2 


            return [lexical_sim_ratio, syntactic_sim_ratio, num_lexical_items, statement2_pos]
    
        except:
            return [0,0,0,0]
        

        
def spacy_semantic_sim(utt_1, utt_2):
    
    return nlp(utt_1).similarity(nlp(utt_2))



def lilla(statement1, statement2):
    
    statement1 =[x.lower().strip() for x in statement1.split()]
    statement2 =[x.lower().strip() for x in statement2.split()]
    
    if (len(statement1) or len(statement2)) == 0:

        return [0,0,0,0]
    
    else:
    
        shared_lexical_items = 0
        num_lexical_items = len(statement1) + len(statement2)

        for word in statement1:
            if word in statement2:
                shared_lexical_items += 1

        lexical_sim_ratio = (shared_lexical_items/num_lexical_items) * 2

        nlp_statement1 = nlp(' '.join(statement1))

        nlp_statement2 = nlp(' '.join(statement2))

        statement1_pos = [x.pos_ for x in nlp_statement1]

        statement2_pos = [x.pos_ for x in nlp_statement2]

        shared_pos = 0

        normalized_pos =  len(statement1_pos) * len(statement2_pos)
        
        try:

            for pos in statement1_pos:
                if pos in statement2_pos:
                    shared_pos += 1

            syntactic_sim_ratio = (shared_pos/normalized_pos) 

            return [lexical_sim_ratio, syntactic_sim_ratio, num_lexical_items, statement2_pos]
        
        except:
            return [0,0,0,0]
        
def ngram_pos(sequence1,sequence2,ngramsize=2,
                   ignore_duplicates=True):
    """
    Remove mimicked lexical sequences from two interlocutors'
    sequences and return a dictionary of counts of ngrams
    of the desired size for each sequence.
    By default, consider bigrams. If desired, this may be
    changed by setting `ngramsize` to the appropriate
    value.
    By default, ignore duplicate lexical n-grams when
    processing these sequences. If desired, this may
    be changed with `ignore_duplicates=False`.
    """

    # remove duplicates and recreate sequences
    sequence1 = set(ngrams(sequence1,ngramsize))
    sequence2 = set(ngrams(sequence2,ngramsize))

    # if desired, remove duplicates from sequences
    if ignore_duplicates:
        new_sequence1 = [tuple([''.join(pair[:]) for pair in tup]) for tup in list(sequence1 - sequence2)]
        new_sequence2 = [tuple([''.join(pair[:]) for pair in tup]) for tup in list(sequence2 - sequence1)]
#         print(new_sequence1, new_sequence2)
    else:
        new_sequence1 = [tuple([''.join(pair[:]) for pair in tup]) for tup in sequence1]
        new_sequence2 = [tuple([''.join(pair[:]) for pair in tup]) for tup in sequence2]

    # return counters
    return Counter(new_sequence1), Counter(new_sequence2)


def ngram_lexical(sequence1,sequence2,ngramsize=2):
    """
    Create ngrams of the desired size for each of two
    interlocutors' sequences and return a dictionary
    of counts of ngrams for each sequence.
    By default, consider bigrams. If desired, this may be
    changed by setting `ngramsize` to the appropriate
    value.
    """

    # generate ngrams
    sequence1 = list(ngrams(sequence1,ngramsize))
    sequence2 = list(ngrams(sequence2,ngramsize))

    # join for counters
    new_sequence1 = [' '.join(pair) for pair in sequence1]
    new_sequence2 = [' '.join(pair) for pair in sequence2]

    # return counters
    return Counter(new_sequence1), Counter(new_sequence2)


def get_cosine(vec1, vec2):
    """
    Derive cosine similarity metric, standard measure.
    Adapted from <https://stackoverflow.com/a/33129724>.
    """

    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x]**2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator



def LexicalPOSAlignment(tok1,lem1,penn_tok1,penn_lem1,
                             tok2,lem2,penn_tok2,penn_lem2,
                             stan_tok1=None,stan_lem1=None,
                             stan_tok2=None,stan_lem2=None,
                             maxngram=2,
                             ignore_duplicates=True,
                             add_stanford_tags=False):

    """
    Derive lexical and part-of-speech alignment scores
    between interlocutors (suffix `1` and `2` in arguments
    passed to function).
    By default, return scores based only on Penn POS taggers.
    If desired, also return scores using Stanford tagger with
    `add_stanford_tags=True` and by providing appropriate
    values for `stan_tok1`, `stan_lem1`, `stan_tok2`, and
    `stan_lem2`.
    By default, consider only bigram when calculating
    similarity. If desired, this window may be expanded
    by changing the `maxngram` argument value.
    By default, remove exact duplicates when calculating
    similarity scores (i.e., does not consider perfectly
    mimicked lexical items between speakers). If desired,
    duplicates may be included when calculating scores by
    passing `ignore_duplicates=False`.
    """

    # create empty dictionaries for syntactic similarity
    syntax_penn_tok = {}
    syntax_penn_lem = {}

    # if desired, generate Stanford-based scores
    if add_stanford_tags:
        syntax_stan_tok = {}
        syntax_stan_lem = {}

    # create empty dictionaries for lexical similarity
    lexical_tok = {}
    lexical_lem = {}

    # cycle through all desired ngram lengths
    for ngram in range(1,maxngram+1):

        # calculate similarity for lexical ngrams (tokens and lemmas)
        [vectorT1, vectorT2] = ngram_lexical(tok1,tok2,ngramsize=ngram)
        [vectorL1, vectorL2] = ngram_lexical(lem1,lem2,ngramsize=ngram)
        lexical_tok['lexical_tok{0}'.format(ngram)] = get_cosine(vectorT1,vectorT2)
        lexical_lem['lexical_lem{0}'.format(ngram)] = get_cosine(vectorL1, vectorL2)

        # calculate similarity for Penn POS ngrams (tokens)
        [vector_penn_tok1, vector_penn_tok2] = ngram_pos(penn_tok1,penn_tok2,
                                                ngramsize=ngram,
                                                ignore_duplicates=ignore_duplicates)
        syntax_penn_tok['syntax_penn_tok{0}'.format(ngram)] = get_cosine(vector_penn_tok1,
                                                                                            vector_penn_tok2)
        # calculate similarity for Penn POS ngrams (lemmas)
        [vector_penn_lem1, vector_penn_lem2] = ngram_pos(penn_lem1,penn_lem2,
                                                              ngramsize=ngram,
                                                              ignore_duplicates=ignore_duplicates)
        syntax_penn_lem['syntax_penn_lem{0}'.format(ngram)] = get_cosine(vector_penn_lem1,
                                                                                            vector_penn_lem2)

        # if desired, also calculate using Stanford POS
        if add_stanford_tags:

            # calculate similarity for Stanford POS ngrams (tokens)
            [vector_stan_tok1, vector_stan_tok2] = ngram_pos(stan_tok1,stan_tok2,
                                                                  ngramsize=ngram,
                                                                  ignore_duplicates=ignore_duplicates)
            syntax_stan_tok['syntax_stan_tok{0}'.format(ngram)] = get_cosine(vector_stan_tok1,
                                                                                                vector_stan_tok2)

            # calculate similarity for Stanford POS ngrams (lemmas)
            [vector_stan_lem1, vector_stan_lem2] = ngram_pos(stan_lem1,stan_lem2,
                                                                  ngramsize=ngram,
                                                                  ignore_duplicates=ignore_duplicates)
            syntax_stan_lem['syntax_stan_lem{0}'.format(ngram)] = get_cosine(vector_stan_lem1,
                                                                                                vector_stan_lem2)

    # return requested information
    if add_stanford_tags:
        dictionaries_list = [syntax_penn_tok, syntax_penn_lem,
                             syntax_stan_tok, syntax_stan_lem,
                             lexical_tok, lexical_lem]
    else:
        dictionaries_list = [syntax_penn_tok, syntax_penn_lem,
                             lexical_tok, lexical_lem]

    return dictionaries_list


    
    
    
    


In [ ]:
### eval alignment trees

df = pd.read_csv('ldp_typical_kids_utts.csv') ### or ldp_atypical_kids_utts.csv

df['p_tree_parse'] = [cont_parse(x) if x != '' else '()' for x in df['p_utts'].fillna('').tolist()]
df['c_tree_parse'] = [cont_parse(x) if x != '' else '()' for x in df['c_utts'].fillna('').tolist()]

df['p_tree_parse_nodes_only'] = df.p_tree_parse.map(lambda x: cont_tree_nodes_only(x))
df['c_tree_parse_nodes_only'] = df.c_tree_parse.map(lambda x: cont_tree_nodes_only(x))

df['p_tree_height'] = df['p_tree_parse_nodes_only'].map(lambda x: tree_depth(x))
df['c_tree_height'] = df['c_tree_parse_nodes_only'].map(lambda x: tree_depth(x))

df['p_tree_preorder_traversal'] = df['p_tree_parse_nodes_only'].map(lambda x: pre_traversal_str(x))
df['c_tree_preorder_traversal'] = df['c_tree_parse_nodes_only'].map(lambda x: pre_traversal_str(x))

df['p_tree_dist_format'] = df['p_tree_parse_nodes_only'].map(lambda x: apt_edit_tree_dist_format(x))
df['c_tree_dist_format'] = df['c_tree_parse_nodes_only'].map(lambda x: apt_edit_tree_dist_format(x))


### get speaker labels ###

p_utts = df.p_utts.fillna('').tolist()
c_utts = df.c_utts.fillna('').tolist()

speaker_label = []

for i in range(len(df)):
    if (p_utts[i] != '') and (c_utts[i] != ''):
        speaker_label.append('both')
        continue
    
    if (p_utts[i] != ''):
        speaker_label.append('p')
        continue
        
    elif (c_utts[i] != ''):
        speaker_label.append('c')
        continue
        
    else:
        speaker_label.append('')

        
df['speaker_label'] = speaker_label


### label speaker shifts

speaker_shift = []
for i in range(len(df)):
    if speaker_label[i] != speaker_label[i-1]:
        speaker_shift.append(True)
    else:
        speaker_shift.append(False)
        
speaker_shift[0] = False ### initialize first utt as false

df['speaker_shift'] = speaker_shift

speaker_shift_li = df.speaker_shift.tolist()
speaker_label_li = df.speaker_label.tolist()
p_tree_dist_format_li = ['{}' if x == '' else x for x in df['p_tree_dist_format'].tolist()]
c_tree_dist_format_li = ['{}' if x == '' else x for x in df['c_tree_dist_format'].tolist()]



edit_distances = []
parent_resps = [False for i in range(len(df))]
child_resps = [False for i in range(len(df))]
align_opportunity = [False for i in range(len(df))]
for i in range(len(df)):
    print(i)
    if speaker_shift_li[i] == True:
        if speaker_label_li[i] == 'c':            
            edit_distances.append(calculate_tree_edit_distance(p_tree_dist_format_li[i-1], c_tree_dist_format_li[i]))
            align_opportunity[i-1] = True
            align_opportunity[i] = True
            child_resps[i] = True
            
            
        else:
            edit_distances.append(calculate_tree_edit_distance(c_tree_dist_format_li[i-1], p_tree_dist_format_li[i]))
            align_opportunity[i-1] = True
            align_opportunity[i] = True
            parent_resps[i] = True
    else:
        edit_distances.append(np.nan)
        

df['tree_edit_distances'] = edit_distances
df['alignment_opportunity'] = align_opportunity
df['child_resp'] = child_resps
df['parent_resp'] = parent_resps

tree_edit_distances_li = df.tree_edit_distances.tolist()
p_utts_parse_tree_dist_format_li = df.p_tree_dist_format.tolist()
c_utts_parse_tree_dist_format_li = df.c_tree_dist_format.tolist()
parent_tree_depth_li = df.p_tree_height.tolist()
child_tree_depth_li = df.c_tree_height.tolist()
parent_pre_order_traversal_li = df.p_tree_preorder_traversal.tolist()
child_pre_order_traversal_li = df.c_tree_preorder_traversal.tolist()

p_utts = df.p_utts.fillna('').tolist() 
c_utts = df.c_utts.fillna('').tolist()
parent_resp_li = df.parent_resp.tolist()
child_resp_li = df.child_resp.tolist()

comp_trees = []
parent_resps = [False for i in range(len(df))]
child_resps = [False for i in range(len(df))]
align_opportunity = [False for i in range(len(df))]
for i in range(len(df)):
    
    if speaker_shift_li[i] == True:
        
        count = 0
        

        if (speaker_label_li[i] == 'c') and (parent_tree_depth_li[i-1] != 0):


            count += 1
            print(count, 'CHILD')
            comp_trees.append(tree_sim_dev(parent_pre_order_traversal_li[i-1], child_pre_order_traversal_li[i], p_utts_parse_tree_dist_format_li[i-1], c_utts_parse_tree_dist_format_li[i], parent_tree_depth_li[i-1], child_tree_depth_li[i], tree_edit_distances_li[i]))

            print(tree_sim_dev(parent_pre_order_traversal_li[i-1], child_pre_order_traversal_li[i], p_utts_parse_tree_dist_format_li[i-1], c_utts_parse_tree_dist_format_li[i], parent_tree_depth_li[i-1], child_tree_depth_li[i], tree_edit_distances_li[i]))

            align_opportunity[i-1] = True
            align_opportunity[i] = True
            child_resps[i] = True

            continue

                
        if (speaker_label_li[i] == 'p') and (child_tree_depth_li[i-1] != 0):
            count += 1
            print(count, 'PARENT')
            print(i, parent_tree_depth_li[i], child_tree_depth_li[i-1])

            print(tree_sim_dev(child_pre_order_traversal_li[i-1], parent_pre_order_traversal_li[i], c_utts_parse_tree_dist_format_li[i-1], p_utts_parse_tree_dist_format_li[i], child_tree_depth_li[i-1], parent_tree_depth_li[i], tree_edit_distances_li[i]))
            comp_trees.append(tree_sim_dev(child_pre_order_traversal_li[i-1], parent_pre_order_traversal_li[i], c_utts_parse_tree_dist_format_li[i-1], p_utts_parse_tree_dist_format_li[i], child_tree_depth_li[i-1], parent_tree_depth_li[i], tree_edit_distances_li[i]))
            align_opportunity[i-1] = True
            align_opportunity[i] = True
            parent_resps[i] = True


        else:
            count += 1
            
            comp_trees.append({})

    else:
        comp_trees.append({})
        
df['tree_comp_dicts'] = comp_trees

df['ldp_tree_sim'] = df['tree_comp_dicts'].map(lambda x: x.get('cosine_sim'))

df['tree_edit_sim'] = df['tree_edit_distances'].map(lambda x: tree_edit_sim(x))
df['tree_edit_sim_sqrt'] = df['tree_edit_distances'].map(lambda x: tree_edit_sim_sqrt(x))

### EXPORT TO CSV ###


df.to_csv('TYPICAL_ldp_children_tree_comparison_data_1_25_22.csv') ### or ATYPICAL



In [ ]:
### read in dfs 

aty_df = pd.read_csv('ATYPICAL_ldp_children_tree_comparison_data_1_28_22.csv')

typ_df = pd.read_csv('TYPICAL_ldp_children_tree_comparison_data_1_25_22.csv')


### find one word c_utts ###

c_utts = typ_df.c_utts.fillna('').tolist()
typ_one_word = []
cs = [i for i, x in enumerate (typ_df.child_resps.tolist())]
for i in cs:
    if len(c_utts[i].strip().split()) == 1:
        typ_one_word.append(True)
    else:
        typ_one_word.append(False)
        
        
c_utts = aty_df.c_utts.fillna('').tolist()
aty_one_word = []
cs = [i for i, x in enumerate (aty_df.child_resps.tolist())]
for i in cs:
    if len(c_utts[i].strip().split()) == 1:
        aty_one_word.append(True)
    else:
        aty_one_word.append(False)
        
        
typ_df['one_word_bool'] = typ_one_word

aty_df['one_word_bool'] = aty_one_word


### TYP / TYPICAL DF ONLY FROM HERE ON FUSAROLI ###


speaker_shifts = typ_df.speaker_shift.tolist()
speaker_labels = typ_df.speaker_label.tolist()
p_utts = typ_df.p_utts.fillna('').tolist()
c_utts = typ_df.c_utts.fillna('').tolist()

fusarolli_align_li = []
pcount = 0
ccount = 0
ncount = 0
syntactic_lexical_sim_li = []
lilla_li  = []

for i, x in enumerate(speaker_shifts):
    if (x == True) and (speaker_labels[i] == 'p'):
        pcount += 1
        syntactic_lexical_sim_li.append(syntactic_lexical_sim(p_utts[i], c_utts[i-1]))
        lilla_li.append(lilla(p_utts[i], c_utts[i-1]))
        prep_data = fusarolli_align_prep(p_utts[i], c_utts[i-1])
        token1 = prep_data[0]
        lemma1 = prep_data[1]
        pos_tok1 =  prep_data[2]
        pos_lemma1 = prep_data[3]
        token2 = prep_data[4]
        lemma2 = prep_data[5]
        pos_tok2 = prep_data[6]
        pos_lemma2 = prep_data[7]
        fusarolli_align_li.append(LexicalPOSAlignment(token1, lemma1, pos_tok1, pos_lemma1, token2, lemma2, pos_tok2, pos_lemma2, ignore_duplicates=False))
        
    
    if (x == True) and (speaker_labels[i] == 'c'):
        ccount += 1
        syntactic_lexical_sim_li.append(syntactic_lexical_sim(c_utts[i], p_utts[i-1]))
        lilla_li.append(lilla(c_utts[i], p_utts[i-1]))
        prep_data = fusarolli_align_prep(c_utts[i], p_utts[i-1])
        token1 = prep_data[0]
        lemma1 = prep_data[1]
        pos_tok1 =  prep_data[2]
        pos_lemma1 = prep_data[3]
        token2 = prep_data[4]
        lemma2 = prep_data[5]
        pos_tok2 = prep_data[6]
        pos_lemma2 = prep_data[7]
        fusarolli_align_li.append(LexicalPOSAlignment(token1, lemma1, pos_tok1, pos_lemma1, token2, lemma2, pos_tok2, pos_lemma2, ignore_duplicates=False))
        
    if (x == True) and (speaker_labels[i] == 'both'):
        ccount += 1
        prep_data = fusarolli_align_prep(c_utts[i], p_utts[i-1])
        token1 = prep_data[0]
        lemma1 = prep_data[1]
        pos_tok1 =  prep_data[2]
        pos_lemma1 = prep_data[3]
        token2 = prep_data[4]
        lemma2 = prep_data[5]
        pos_tok2 = prep_data[6]
        pos_lemma2 = prep_data[7]
        fusarolli_align_li.append(LexicalPOSAlignment(token1, lemma1, pos_tok1, pos_lemma1, token2, lemma2, pos_tok2, pos_lemma2, ignore_duplicates=False))
        
    else:
        syntactic_lexical_sim_li.append([])
        lilla_li.append([])
        fusarolli_align_li.append([])
        ncount += 1 
        
        
typ_df['syntactic_lexical_sim_li'] = syntactic_lexical_sim_li
typ_df['lilla_li'] = lilla_li
typ_df['fusarolli_align_li'] = fusarolli_align_li



### EXPORT TO CSV

typ_df.to_csv('typ_df.csv')



In [ ]:
### ATYP / ATYPICAL DF ONLY FROM HERE ON FUSAROLI


speaker_shifts = aty_df.speaker_shift.tolist()
speaker_labels = aty_df.speaker_label.tolist()
p_utts = aty_df.p_utts.fillna('').tolist()
c_utts = aty_df.c_utts.fillna('').tolist()
parent_tree_depth_li = aty_df.p_tree_height.tolist()
child_tree_depth_li = aty_df.c_tree_height.tolist()


fusarolli_align_li = []
pcount = 0
ccount = 0
ncount = 0
syntactic_lexical_sim_li = []
lilla_li  = []

for i in range(len(aty_df)):
    
    if speaker_shifts[i] == True:
        
        count = 0

        if (speaker_labels[i] == 'c') and (parent_tree_depth_li[i-1] != 0):
        
            syntactic_lexical_sim_li.append(syntactic_lexical_sim(p_utts[i-1], c_utts[i]))
            lilla_li.append(lilla(p_utts[i-1], c_utts[i]))
            prep_data = fusarolli_align_prep(p_utts[i], c_utts[i])
            token1 = prep_data[0]
            lemma1 = prep_data[1]
            pos_tok1 =  prep_data[2]
            pos_lemma1 = prep_data[3]
            token2 = prep_data[4]
            lemma2 = prep_data[5]
            pos_tok2 = prep_data[6]
            pos_lemma2 = prep_data[7]
            fusarolli_align_li.append(LexicalPOSAlignment(token1, lemma1, pos_tok1, pos_lemma1, token2, lemma2, pos_tok2, pos_lemma2, ignore_duplicates=False))

            continue

                
        if (speaker_labels[i] == 'p') and (child_tree_depth_li[i-1] != 0):
            
            syntactic_lexical_sim_li.append(syntactic_lexical_sim(c_utts[i-1], p_utts[i]))
            lilla_li.append(lilla(c_utts[i-1], p_utts[i]))
            prep_data = fusarolli_align_prep(c_utts[i-1], p_utts[i])
            token1 = prep_data[0]
            lemma1 = prep_data[1]
            pos_tok1 =  prep_data[2]
            pos_lemma1 = prep_data[3]
            token2 = prep_data[4]
            lemma2 = prep_data[5]
            pos_tok2 = prep_data[6]
            pos_lemma2 = prep_data[7]
            fusarolli_align_li.append(LexicalPOSAlignment(token1, lemma1, pos_tok1, pos_lemma1, token2, lemma2, pos_tok2, pos_lemma2, ignore_duplicates=False))

        else:
            fusarolli_align_li.append([])
            syntactic_lexical_sim_li.append([])
            lilla_li.append([])

    else:
        
        fusarolli_align_li.append([])
        syntactic_lexical_sim_li.append([])
        lilla_li.append([])
        
aty_df['syntactic_lexical_sim_li'] = syntactic_lexical_sim_li
aty_df['lilla_li'] = lilla_li
aty_df['fusarolli_align_li'] = fusarolli_align_li


### EXPORT TO CSV

typ_df.to_csv('aty_df.csv')


In [ ]:

aty_df['ldp_lexical_sim'] = [x[0] if len(x) != 0 else np.nan for x in aty_df['lilla_li'].tolist()] 
aty_df['ldp_syntactic_sim']  = [x[1] if len(x) != 0 else np.nan for x in aty_df['lilla_li'].tolist()] 

aty_df['fusarolli_align_syntax_penn_tok1'] = [x[0].get('syntax_penn_tok1') if len(x) != 0 else np.nan for x in aty_df['fusarolli_align_li'].tolist()] 
aty_df['fusarolli_align_syntax_penn_tok2']  = [x[0].get('syntax_penn_tok2') if len(x) != 0 else np.nan for x in aty_df['fusarolli_align_li'].tolist()] 

aty_df['fusarolli_align_syntax_penn_lemma1'] = [x[1].get('syntax_penn_lem1') if len(x) != 0 else np.nan for x in aty_df['fusarolli_align_li'].tolist()] 
aty_df['fusarolli_align_syntax_penn_lemma2']  = [x[1].get('syntax_penn_lem2') if len(x) != 0 else np.nan for x in aty_df['fusarolli_align_li'].tolist()] 

aty_df['fusarolli_align_lexical_penn_tok1'] = [x[2].get('lexical_tok1') if len(x) != 0 else np.nan for x in aty_df['fusarolli_align_li'].tolist()] 
aty_df['fusarolli_align_lexical_penn_tok2']  = [x[2].get('lexical_tok2') if len(x) != 0 else np.nan for x in aty_df['fusarolli_align_li'].tolist()] 

aty_df['fusarolli_align_lexical_penn_lemma1'] = [x[3].get('lexical_lem1') if len(x) != 0 else np.nan for x in aty_df['fusarolli_align_li'].tolist()] 
aty_df['fusarolli_align_lexical_penn_lemma2']  = [x[3].get('lexical_lem2') if len(x) != 0 else np.nan for x in aty_df['fusarolli_align_li'].tolist()] 

aty_df['ldp_lexical_sim_OLD'] = [x[0] if len(x) != 0 else np.nan for x in aty_df['syntactic_lexical_sim_li'].tolist()] 
aty_df['ldp_syntactic_sim_OLD']  = [x[1] if len(x) != 0 else np.nan for x in aty_df['syntactic_lexical_sim_li'].tolist()] 

typ_df['ldp_lexical_sim'] = [x[0] if len(x) != 0 else np.nan for x in typ_df['lilla_li'].tolist()] 
typ_df['ldp_syntactic_sim']  = [x[1] if len(x) != 0 else np.nan for x in typ_df['lilla_li'].tolist()]

typ_df['ldp_lexical_sim'] = [x[0] if len(x) != 0 else np.nan for x in typ_df['lilla_li'].tolist()] 
typ_df['ldp_syntactic_sim']  = [x[1] if len(x) != 0 else np.nan for x in typ_df['lilla_li'].tolist()] 

typ_df['fusarolli_align_syntax_penn_tok1'] = [x[0].get('syntax_penn_tok1') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 
typ_df['fusarolli_align_syntax_penn_tok2']  = [x[0].get('syntax_penn_tok2') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 

typ_df['fusarolli_align_syntax_penn_lemma1'] = [x[1].get('syntax_penn_lem1') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 
typ_df['fusarolli_align_syntax_penn_lemma2']  = [x[1].get('syntax_penn_lem2') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 

typ_df['fusarolli_align_lexical_penn_tok1'] = [x[2].get('lexical_tok1') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 
typ_df['fusarolli_align_lexical_penn_tok2']  = [x[2].get('lexical_tok2') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 

typ_df['fusarolli_align_lexical_penn_lemma1'] = [x[3].get('lexical_lem1') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 
typ_df['fusarolli_align_lexical_penn_lemma2']  = [x[3].get('lexical_lem2') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 

typ_df['ldp_lexical_sim_OLD'] = [x[0] if len(x) != 0 else np.nan for x in typ_df['syntactic_lexical_sim_li'].tolist()] 
typ_df['ldp_syntactic_sim_OLD']  = [x[1] if len(x) != 0 else np.nan for x in typ_df['syntactic_lexical_sim_li'].tolist()]

typ_df['fusarolli_align_lexical_penn_tok1'] = [x[2].get('lexical_tok1') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 
typ_df['fusarolli_align_lexical_penn_tok2']  = [x[2].get('lexical_tok2') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 

typ_df['fusarolli_align_lexical_penn_lemma1'] = [x[3].get('lexical_lem1') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 
typ_df['fusarolli_align_lexical_penn_lemma2']  = [x[3].get('lexical_lem2') if len(x) != 0 else np.nan for x in typ_df['fusarolli_align_li'].tolist()] 


In [ ]:
### EXPORT TO CSVS

typ_df.to_csv('ldp_typical_children_tree_comparison_w_ldp_and_fusarolli_lexical_syntactic_alignment_1_30_2022.csv')

aty_df.to_csv('ldp_atypical_children_tree_comparison_w_ldp_and_fusarolli_lexical_syntactic_alignment_1_30_2022.csv')


In [ ]:
permuted_rand_baseline_resp_avgs = {}

count = 0

for sub in aty_subs:
    permuted_rand_baseline_resp_avgs[sub] = {}
    for s in aty_sessions:
        permuted_rand_baseline_resp_avgs[sub][s] = {}
        
        count += 1
        
    
        cp_fusarolli_align_li = []
        pc_fusarolli_align_li = []
        print(sub, s)
        p_rand_utts = get_p_resp_utts[sub][s]['p_rand_resp_utts']
        c_rand_utts = get_c_resp_utts[sub][s]['c_rand_resp_utts']

        p_rand_utts = [x for x in p_rand_utts if x != '']
        c_rand_utts = [x for x in c_rand_utts if x != '']

        if (len(p_rand_utts) == 0) or (len(c_rand_utts) == 0):
            pass




        else:
            if len(p_rand_utts) > len(c_rand_utts):
                lim = len(c_rand_utts)

            else:
                lim = len(p_rand_utts)

            child_cont_trees = [cont_parse(x) for x in c_rand_utts]
            parent_cont_trees = [cont_parse(x) for x in p_rand_utts]
            parent_tree_depths = [tree_depth(x) for x in parent_cont_trees]
            child_tree_depths = [tree_depth(x) for x in child_cont_trees]
            child_tree_edit_format = [apt_edit_tree_dist_format(cont_tree_nodes_only(x)) for x in child_cont_trees]
            parent_tree_edit_format = [apt_edit_tree_dist_format(cont_tree_nodes_only(x)) for x in parent_cont_trees]

            print(len(child_tree_edit_format), len(parent_tree_edit_format))

            parent_child_edit_dists = [calculate_tree_edit_distance(child_tree_edit_format[i], parent_tree_edit_format[i-1]) for i in range(lim)]
            child_parent_edit_dists = [calculate_tree_edit_distance(parent_tree_edit_format[i], child_tree_edit_format[i-1]) for i in range(lim)]

            ### tree edit_avgs ### 

            print(len(child_parent_edit_dists), len(p_rand_utts), len(c_rand_utts))

            child_parent_tree_edit_sim_sqrt_ave = sum([tree_edit_sim_sqrt(x) for x in child_parent_edit_dists])/lim
            parent_child_tree_edit_sim_sqrt_ave = sum([tree_edit_sim_sqrt(x) for x in parent_child_edit_dists])/lim
            child_parent_tree_edit_sim_ave = sum([tree_edit_sim(x) for x in child_parent_edit_dists])/lim
            parent_child_tree_edit_sim_ave = sum([tree_edit_sim(x) for x in parent_child_edit_dists])/lim

            ### end tree edit avgs ###

            ### start pre_traversal analysis ###

            parent_preorder_traversals = [pre_traversal_str(cont_tree_nodes_only(x)) for x in parent_cont_trees]
            child_preorder_traversals = [pre_traversal_str(cont_tree_nodes_only(x)) for x in child_cont_trees]

            tree_sim_dev_child_parent = sum([tree_sim_dev(child_preorder_traversals[i], parent_preorder_traversals[i], child_tree_edit_format[i], parent_tree_edit_format[i], child_tree_depths[i], parent_tree_depths[i], parent_child_edit_dists[i])['cosine_sim'] for i in range(lim)])/lim

            tree_sim_dev_parent_child = sum([tree_sim_dev(parent_preorder_traversals[i], child_preorder_traversals[i], parent_tree_edit_format[i], child_tree_edit_format[i], parent_tree_depths[i], child_tree_depths[i], child_parent_edit_dists[i])['cosine_sim'] for i in range(lim)])/lim

            print(child_parent_tree_edit_sim_sqrt_ave, parent_child_tree_edit_sim_sqrt_ave, child_parent_tree_edit_sim_ave, parent_child_tree_edit_sim_ave)

            print(tree_sim_dev_child_parent, tree_sim_dev_parent_child)



            ### end pre_traversal analysis ###




            ### start fusarolli ldp syntax, lexical, semantic data analysis ###


            child_parent_lilla = [lilla(c_rand_utts[i], p_rand_utts[i-1]) for i in range(lim)]
            parent_child_lilla = [lilla(p_rand_utts[i], c_rand_utts[i-1]) for i in range(lim)]

            child_parent_old_ldp = [syntactic_lexical_sim(c_rand_utts[i], p_rand_utts[i-1]) for i in range(lim)]

            parent_child_old_ldp = [syntactic_lexical_sim(p_rand_utts[i], c_rand_utts[i-1]) for i in range(lim)]




            child_parent_lilla_li_lex = [x[0] for x in child_parent_lilla]
            parent_child_lilla_li_lex = [x[0] for x in parent_child_lilla]

            child_parent_lilla_li_syntax = [x[1] for x in child_parent_lilla]
            parent_child_lilla_li_syntax = [x[1] for x in parent_child_lilla]

            child_parent_lilla_li_lex_ave = sum(child_parent_lilla_li_lex)/lim

            parent_child_lilla_li_lex_ave = sum(parent_child_lilla_li_lex)/lim

            child_parent_lilla_li_syntax_ave = sum(child_parent_lilla_li_syntax)/lim

            parent_child_lilla_li_syntax_ave = sum(parent_child_lilla_li_syntax)/lim


            child_parent_old_ldp_li_lex = [x[0] for x in child_parent_old_ldp] 
            parent_child_old_ldp_li_lex = [x[0] for x in parent_child_old_ldp]

            child_parent_old_ldp_li_syntax = [x[1] for x in child_parent_old_ldp] 
            parent_child_old_ldp_li_syntax = [x[1] for x in parent_child_old_ldp]

            child_parent_old_ldp_li_lex_ave = sum(child_parent_old_ldp_li_lex)/lim


            parent_child_old_ldp_li_lex_ave = sum(parent_child_old_ldp_li_lex)/lim

            child_parent_old_ldp_li_syntax_ave = sum(child_parent_old_ldp_li_syntax)/lim

            parent_child_old_ldp_li_syntax_ave = sum(parent_child_old_ldp_li_syntax)/lim

            print(child_parent_lilla_li_lex_ave, parent_child_lilla_li_lex_ave)

            print(child_parent_lilla_li_syntax_ave, parent_child_lilla_li_syntax_ave)

            print(child_parent_old_ldp_li_lex_ave, parent_child_old_ldp_li_lex_ave)

            print(child_parent_old_ldp_li_syntax_ave, parent_child_old_ldp_li_syntax_ave)




            child_parent_prep_data = [fusarolli_align_prep(p_rand_utts[i], c_rand_utts[i-1]) for i in range(lim)]
            parent_child_prep_data = [fusarolli_align_prep(c_rand_utts[i], p_rand_utts[i-1]) for i in range(lim)]

            cp_token1 = [x[0] for x in child_parent_prep_data]
            cp_lemma1 = [x[1] for x in child_parent_prep_data]
            cp_pos_tok1 = [x[2] for x in child_parent_prep_data]
            cp_pos_lemma1 = [x[3] for x in child_parent_prep_data]
            cp_token2 = [x[4] for x in child_parent_prep_data]
            cp_lemma2 = [x[5] for x in child_parent_prep_data]
            cp_pos_tok2 = [x[6] for x in child_parent_prep_data]
            cp_pos_lemma2 = [x[7] for x in child_parent_prep_data]

            cp_fusarolli_align_li = [LexicalPOSAlignment(cp_token1[i], cp_lemma1[i], cp_pos_tok1[i], cp_pos_lemma1[i], cp_token2[i], cp_lemma2[i], cp_pos_tok2[i], cp_pos_lemma2[i], ignore_duplicates=False) for i in range(lim)]

            cp_fusarolli_align_syntax_penn_tok1_ave = sum([x[0].get('syntax_penn_tok1') if len(x) != 0 else np.nan for x in cp_fusarolli_align_li])/len(cp_fusarolli_align_li) 
            cp_fusarolli_align_syntax_penn_tok2_ave = sum([x[0].get('syntax_penn_tok2') if len(x) != 0 else np.nan for x in cp_fusarolli_align_li]) /len(cp_fusarolli_align_li) 

            cp_fusarolli_align_syntax_penn_lemma1_ave = sum([x[1].get('syntax_penn_lem1') if len(x) != 0 else np.nan for x in cp_fusarolli_align_li])/len(cp_fusarolli_align_li)
            cp_fusarolli_align_syntax_penn_lemma2_ave  = sum([x[1].get('syntax_penn_lem2') if len(x) != 0 else np.nan for x in cp_fusarolli_align_li]) / len(cp_fusarolli_align_li)

            cp_fusarolli_align_lexical_penn_tok1_ave = sum([x[2].get('lexical_tok1') if len(x) != 0 else np.nan for x in cp_fusarolli_align_li]) / len(cp_fusarolli_align_li)
            cp_fusarolli_align_lexical_penn_tok2_ave  = sum([x[2].get('lexical_tok2') if len(x) != 0 else np.nan for x in cp_fusarolli_align_li]) / len(cp_fusarolli_align_li)

            cp_fusarolli_align_lexical_penn_lemma1_ave = sum([x[3].get('lexical_lem1') if len(x) != 0 else np.nan for x in cp_fusarolli_align_li]) / len(cp_fusarolli_align_li)
            cp_fusarolli_align_lexical_penn_lemma2_ave = sum([x[3].get('lexical_lem2') if len(x) != 0 else np.nan for x in cp_fusarolli_align_li]) / len(cp_fusarolli_align_li)



            pc_token1 = [x[0] for x in parent_child_prep_data]
            pc_lemma1 = [x[1] for x in parent_child_prep_data]
            pc_pos_tok1 = [x[2] for x in parent_child_prep_data]
            pc_pos_lemma1 = [x[3] for x in parent_child_prep_data]
            pc_token2 = [x[4] for x in parent_child_prep_data]
            pc_lemma2 = [x[5] for x in parent_child_prep_data]
            pc_pos_tok2 = [x[6] for x in parent_child_prep_data]
            pc_pos_lemma2 = [x[7] for x in parent_child_prep_data]

            pc_fusarolli_align_li = [LexicalPOSAlignment(pc_token1[i], pc_lemma1[i], pc_pos_tok1[i], pc_pos_lemma1[i], pc_token2[i], pc_lemma2[i], pc_pos_tok2[i], pc_pos_lemma2[i], ignore_duplicates=False) for i in range(lim)]



            pc_fusarolli_align_syntax_penn_tok1_ave = sum([x[0].get('syntax_penn_tok1') if len(x) != 0 else np.nan for x in pc_fusarolli_align_li])/len(pc_fusarolli_align_li) 
            pc_fusarolli_align_syntax_penn_tok2_ave = sum([x[0].get('syntax_penn_tok2') if len(x) != 0 else np.nan for x in pc_fusarolli_align_li]) /len(pc_fusarolli_align_li) 

            pc_fusarolli_align_syntax_penn_lemma1_ave = sum([x[1].get('syntax_penn_lem1') if len(x) != 0 else np.nan for x in pc_fusarolli_align_li])/len(pc_fusarolli_align_li)
            pc_fusarolli_align_syntax_penn_lemma2_ave  = sum([x[1].get('syntax_penn_lem2') if len(x) != 0 else np.nan for x in pc_fusarolli_align_li]) / len(pc_fusarolli_align_li)

            pc_fusarolli_align_lexical_penn_tok1_ave = sum([x[2].get('lexical_tok1') if len(x) != 0 else np.nan for x in pc_fusarolli_align_li]) / len(pc_fusarolli_align_li)
            pc_fusarolli_align_lexical_penn_tok2_ave  = sum([x[2].get('lexical_tok2') if len(x) != 0 else np.nan for x in pc_fusarolli_align_li]) / len(pc_fusarolli_align_li)

            pc_fusarolli_align_lexical_penn_lemma1_ave = sum([x[3].get('lexical_lem1') if len(x) != 0 else np.nan for x in pc_fusarolli_align_li]) / len(pc_fusarolli_align_li)
            pc_fusarolli_align_lexical_penn_lemma2_ave = sum([x[3].get('lexical_lem2') if len(x) != 0 else np.nan for x in pc_fusarolli_align_li]) / len(pc_fusarolli_align_li)

            print('FUSAROLI')

            print( pc_fusarolli_align_syntax_penn_tok1_ave,  
            pc_fusarolli_align_syntax_penn_tok2_ave,
            pc_fusarolli_align_syntax_penn_lemma1_ave,
            pc_fusarolli_align_syntax_penn_lemma2_ave, 
            pc_fusarolli_align_lexical_penn_tok1_ave,
            pc_fusarolli_align_lexical_penn_tok2_ave,
            pc_fusarolli_align_lexical_penn_lemma1_ave, 
            pc_fusarolli_align_lexical_penn_lemma2_ave,

            cp_fusarolli_align_syntax_penn_tok1_ave,  
            cp_fusarolli_align_syntax_penn_tok2_ave,
            cp_fusarolli_align_syntax_penn_lemma1_ave,
            cp_fusarolli_align_syntax_penn_lemma2_ave, 
            cp_fusarolli_align_lexical_penn_tok1_ave,
            cp_fusarolli_align_lexical_penn_tok2_ave,
            cp_fusarolli_align_lexical_penn_lemma1_ave, 
            cp_fusarolli_align_lexical_penn_lemma2_ave)

            ## end fusarolli ldp syntax, lexical, semantic data analysis ###


            ### start semantic simalirity scores ###

            rand_semantic_sim_ave = sum([nlp(c_rand_utts[i]).similarity(nlp(p_rand_utts[i])) for i in range(lim)])/lim

            ### end semantic similarity scores ###


            #### ADD to dictionary ####

            permuted_rand_baseline_resp_avgs[sub][s]['child_parent_tree_edit_sim_sqrt_ave'] = child_parent_tree_edit_sim_sqrt_ave 
            permuted_rand_baseline_resp_avgs[sub][s]['parent_child_tree_edit_sim_sqrt_ave'] = parent_child_tree_edit_sim_sqrt_ave 
            permuted_rand_baseline_resp_avgs[sub][s]['child_parent_tree_edit_sim_ave'] = child_parent_tree_edit_sim_ave
            permuted_rand_baseline_resp_avgs[sub][s]['parent_child_tree_edit_sim_ave'] = parent_child_tree_edit_sim_ave

            permuted_rand_baseline_resp_avgs[sub][s]['tree_sim_dev_child_parent'] = tree_sim_dev_child_parent
            permuted_rand_baseline_resp_avgs[sub][s]['tree_sim_dev_parent_child'] = tree_sim_dev_parent_child

            permuted_rand_baseline_resp_avgs[sub][s]['child_parent_lilla_li_lex_ave'] = child_parent_lilla_li_lex_ave
            permuted_rand_baseline_resp_avgs[sub][s]['parent_child_lilla_li_lex_ave'] = parent_child_lilla_li_lex_ave

            permuted_rand_baseline_resp_avgs[sub][s]['child_parent_lilla_li_syntax_ave'] = child_parent_lilla_li_syntax_ave
            permuted_rand_baseline_resp_avgs[sub][s]['parent_child_lilla_li_syntax_ave'] = parent_child_lilla_li_syntax_ave

            permuted_rand_baseline_resp_avgs[sub][s]['child_parent_old_ldp_li_lex_ave'] = child_parent_old_ldp_li_lex_ave
            permuted_rand_baseline_resp_avgs[sub][s]['parent_child_old_ldp_li_lex_ave'] = parent_child_old_ldp_li_lex_ave

            permuted_rand_baseline_resp_avgs[sub][s]['child_parent_old_ldp_li_syntax_ave'] = child_parent_old_ldp_li_syntax_ave
            permuted_rand_baseline_resp_avgs[sub][s]['parent_child_old_ldp_li_syntax_ave'] = parent_child_old_ldp_li_syntax_ave



            permuted_rand_baseline_resp_avgs[sub][s]['pc_fusarolli_align_syntax_penn_tok1_ave'] = pc_fusarolli_align_syntax_penn_tok1_ave,  
            permuted_rand_baseline_resp_avgs[sub][s]['pc_fusarolli_align_syntax_penn_tok2_ave'] = pc_fusarolli_align_syntax_penn_tok2_ave,
            permuted_rand_baseline_resp_avgs[sub][s]['pc_fusarolli_align_syntax_penn_lemma1_ave'] = pc_fusarolli_align_syntax_penn_lemma1_ave,
            permuted_rand_baseline_resp_avgs[sub][s]['pc_fusarolli_align_syntax_penn_lemma2_ave'] = pc_fusarolli_align_syntax_penn_lemma2_ave, 
            permuted_rand_baseline_resp_avgs[sub][s]['pc_fusarolli_align_lexical_penn_tok1_ave']  = pc_fusarolli_align_lexical_penn_tok1_ave,
            permuted_rand_baseline_resp_avgs[sub][s]['pc_fusarolli_align_lexical_penn_tok2_ave']  = pc_fusarolli_align_lexical_penn_tok2_ave,
            permuted_rand_baseline_resp_avgs[sub][s]['pc_fusarolli_align_lexical_penn_lemma1_ave'] =pc_fusarolli_align_lexical_penn_lemma1_ave, 
            permuted_rand_baseline_resp_avgs[sub][s]['pc_fusarolli_align_lexical_penn_lemma2_ave'] = pc_fusarolli_align_lexical_penn_lemma2_ave,

            permuted_rand_baseline_resp_avgs[sub][s]['cp_fusarolli_align_syntax_penn_tok1_ave'] = cp_fusarolli_align_syntax_penn_tok1_ave 
            permuted_rand_baseline_resp_avgs[sub][s]['cp_fusarolli_align_syntax_penn_tok2_ave'] = cp_fusarolli_align_syntax_penn_tok2_ave
            permuted_rand_baseline_resp_avgs[sub][s]['cp_fusarolli_align_syntax_penn_lemma1_ave'] = cp_fusarolli_align_syntax_penn_lemma1_ave
            permuted_rand_baseline_resp_avgs[sub][s]['cp_fusarolli_align_syntax_penn_lemma2_ave'] = cp_fusarolli_align_syntax_penn_lemma2_ave 
            permuted_rand_baseline_resp_avgs[sub][s]['cp_fusarolli_align_lexical_penn_tok1_ave'] = cp_fusarolli_align_lexical_penn_tok1_ave
            permuted_rand_baseline_resp_avgs[sub][s]['cp_fusarolli_align_lexical_penn_tok2_ave'] = cp_fusarolli_align_lexical_penn_tok2_ave
            permuted_rand_baseline_resp_avgs[sub][s]['cp_fusarolli_align_lexical_penn_lemma1_ave'] = cp_fusarolli_align_lexical_penn_lemma1_ave
            permuted_rand_baseline_resp_avgs[sub][s]['cp_fusarolli_align_lexical_penn_lemma2_ave'] = cp_fusarolli_align_lexical_penn_lemma2_ave
            permuted_rand_baseline_resp_avgs[sub][s]['rand_semantic_sim_ave'] = rand_semantic_sim_ave

            ###

            print(permuted_rand_baseline_resp_avgs)
